# MNIST digit classifier example

#### Imports

In [1]:
%matplotlib inline

import numpy as np

from matplotlib import pyplot as plt

import sklearn.cross_validation

import lasagne
import theano.tensor as T
from britefury_lasagne import basic_dnn, trainer, image_window_extractor, mnist



EVIL HACK: Disable cuDNN check


DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpkzwrpq/265abc51f7c376c224983485238ff1a5.lib and object D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpkzwrpq/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 25.0% of memory, cuDNN 5103)
d:\packages\theano\theano\sandbox\cuda\__init__.py:603: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### Define network architecture

We define the `build_network` function that takes the input variables as an optional argument and build the network using the Lasagne API.

NOTE that the final dense layer does *NOT* use the `softmax` nonlinearity as it is supplied by the classifier builder (see below).

In [2]:
def build_network(input_vars=None):
    # Input layer
    x_var = input_vars[0] if input_vars is not None else None
    net = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=x_var)

    # A 32 unit 5x5 conv layer, followed by 2x2 max-pool
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(5, 5), W=lasagne.init.HeUniform())
    net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2, 2))

    # Two 32 unit 3x3 conv layers, followed by 2x2 max-pool
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3, 3), W=lasagne.init.HeUniform())
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3, 3), W=lasagne.init.HeUniform())
    net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2, 2))

    # A fully-connected layer of 64 units followed by 50% dropout
    net = lasagne.layers.DenseLayer(net, num_units=64, W=lasagne.init.HeUniform())
    net = lasagne.layers.DropoutLayer(net, p=0.5)

    # Final 10-unit dense layer, with no nonlinearity
    net = lasagne.layers.DenseLayer(net, num_units=10, nonlinearity=None)

    return net

#### Load the train, validation and test sets

In [3]:
dataset = mnist.MNIST()


### Train the classifier

In [4]:
# Build the image classifier for the given model builder
print('Building network')
clf = basic_dnn.simple_classifier(build_network, n_input_spatial_dims=2,
            updates_fn=lambda loss, params: lasagne.updates.adam(loss, params, learning_rate=0.001))

# Set verbosity
clf.trainer.report(verbosity=trainer.VERBOSITY_EPOCH)

# Set training length
clf.trainer.train_for(num_epochs=10)

# Train
print('Training')
clf.trainer.train([dataset.train_X, dataset.train_y],
                  [dataset.val_X, dataset.val_y],
                  [dataset.test_X, dataset.test_y], batchsize=128)

Building network


DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmp8e7trc/d930a3c0ed413ddeb2464cb6a8818842.lib and object D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmp8e7trc/d930a3c0ed413ddeb2464cb6a8818842.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpvd8z9c/8cffb19b7b08293a4f3e1dbb9c21ddff.lib and object D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpvd8z9c/8cffb19b7b08293a4f3e1dbb9c21ddff.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpnizmkq/e8da842e90e44de224f835542fc1e2ae.lib and object D:/temp/theano/compiledir_Windows-10-10.0.14393-Inte

Training


AttributeError: 'BasicClassifierDNN' object has no attribute 'score_objective'

### Use the classifier to predict on the test set

In [ ]:
# Predict probabilities for test samples
test_y_pred_prob = clf.predict([dataset.test_X])[0]
# Use `np.argmax` to get class predictions
test_y_pred = np.argmax(test_y_pred_prob, axis=1)

# Show the error rate
print('Test error rate={:.2%}'.format(np.mean(test_y_pred != dataset.test_y)))